In [1]:
import torch
import torch.nn.functional as f
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
names = open(r"C:\Users\dageo\Desktop\Names_Makemore.txt", "r").read().splitlines()
train, testing = train_test_split(names, train_size = 0.8,test_size=0.2, random_state=42)
test, dev = train_test_split(testing, train_size = 0.5,test_size=0.5, random_state=42)

In [3]:
chars = sorted(list(set(''.join(names))))

In [4]:
stoi = {ch:i+1 for i,ch in enumerate(chars)}
itos = {v: k for k, v in stoi.items()}
stoi["."] = 0
itos[0] = "."

In [5]:
X,Y = [],[]

block_size = 3

for name in train[:]:
    context = [[0]*block_size]
    add = context[0]
    X.append(add)
    n = (name+".")
    Y.append(stoi[n[0]])
    for i in range(len(n)-1):
        add = context[-1][1:]
        add.append(stoi[n[i]])
        X.append(add)
        context.append(add)
        Y.append(stoi[n[i+1]])

X = torch.tensor(X)
Y = torch.tensor(Y)

In [6]:
X.shape

torch.Size([182497, 3])

In [7]:
Y.shape

torch.Size([182497])

In [8]:
feature_size = 8
tanhlayer_size = 100

In [9]:
C = torch.randn(27,feature_size)

In [10]:
W1 = torch.randn(feature_size*block_size,tanhlayer_size) * 0.1
b1 = torch.randn(tanhlayer_size)*0.01

In [11]:
W2 = torch.randn(tanhlayer_size,feature_size*block_size) * 0.1
b2 = torch.randn(27) * 0.01
parameters = [C, W1, W2, b1, b2]

In [12]:
"""
logits = h @ W2 + b2
counts = logits.exp()
prob = counts/counts.sum(dim = 1, keepdims=True)
"""

'\nlogits = h @ W2 + b2\ncounts = logits.exp()\nprob = counts/counts.sum(dim = 1, keepdims=True)\n'

In [13]:
lrs = torch.linspace(0, -3, 1000)
lre = 10**lrs

In [14]:
#requires grad
for p in parameters:
    p.requires_grad = True

In [15]:
lossi = []

In [16]:
"""
l = []
learn = []
"""
batch_size = 32
steps = 100000
for i in range(steps):
    #forward pass
    #construct minibatch
    ix = torch.randint(0, X.shape[0], (batch_size,))
    emb = C[X[ix]]
    h = torch.tanh(emb.view(emb.shape[0],-1) @ W1 + b1)
    
    #construct logits
    print((h@W2).shape)
    logits = h @ W2 + b2
    loss = f.cross_entropy(logits, Y[ix])
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    lossi.append(loss.log10().item())
    lr = 0.1 if i < 20000 else 0.01
    for p in parameters:
        p.data += -lr*p.grad
    if(i%1000==0):
        print(str(i) + "/" + str(steps) + ": " + str(loss.item()))
        """
        p.data += -lre[i]*p.grad
        l.append(loss.item())
        learn.append(lrs[i])
        """

torch.Size([32, 24])


RuntimeError: The size of tensor a (24) must match the size of tensor b (27) at non-singleton dimension 1

In [ ]:
#checking for oversaturated tanh (The output is 1 or -1 so there is minimal learning). White is a very active tanh neuron
#completely white column is a dead neuron since no matter what is fed into the neuron gets set to 1
#dead neuron never learns since the gradient is 0

plt.figure(figsize = (20,10))
plt.imshow(h.abs()>0.99,cmap = 'gray', interpolation = 'nearest')

In [ ]:
plt.plot(lossi)

In [ ]:
emb = C[X]
h = torch.tanh(emb.view(-1,24) @ W1 + b1)
logits = h @ W2 + b2
loss = f.cross_entropy(logits, Y)

In [ ]:
print(loss)

In [61]:
X_dev,Y_dev = [],[]

block_size = 3

for name in dev[:]:
    context = [[0]*block_size]
    add = context[0]
    X_dev.append(add)
    n = (name+".")
    Y_dev.append(stoi[n[0]])
    for i in range(len(n)-1):
        add = context[-1][1:]
        add.append(stoi[n[i]])
        X_dev.append(add)
        context.append(add)
        Y_dev.append(stoi[n[i+1]])

X_dev = torch.tensor(X_dev)
Y_dev = torch.tensor(Y_dev)

In [62]:
emb = C[X_dev]
h = torch.tanh(emb.view(-1,24) @ W1 + b1)
logits = h @ W2 + b2
loss = f.cross_entropy(logits, Y_dev)

In [63]:
loss

tensor(2.1600, grad_fn=<NllLossBackward0>)

In [66]:
gen_names = []
for _ in range(100):
    out = []
    context = [0]*block_size
    while True:
        emb = C[torch.tensor(context)]
        h = torch.tanh(emb.view(1,-1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = torch.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples = 1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if(ix == 0):
            break
    print(''.join(itos[i] for i in out))
    names.append(''.join(itos[i] for i in out))

irca.
kaurd.
zane.
esmeri.
han.
keu.
vivia.
mana.
baisia.
azarie.
demahile.
domon.
ozmana.
majersiah.
yaniel.
daylanish.
fla.
renlee.
waverick.
raily.
terrion.
lyjahnevela.
elianishand.
elynn.
yam.
khamirielo.
joumi.
dreuge.
hudoden.
jamabrii.
mellani.
bran.
roano.
jous.
ahmarie.
sakidit.
belke.
sarin.
hah.
nale.
jadon.
hed.
karrah.
salennet.
arson.
arth.
inna.
samari.
ruuta.
chazana.
asshalianaelaif.
nice.
fostandleeleyn.
katite.
javi.
rowaristive.
sadi.
loly.
keo.
dechim.
kenylener.
aabroy.
tisniyah.
hude.
anna.
rayan.
reages.
kuna.
ala.
son.
mulle.
juen.
lakimynn.
araven.
amakee.
zhendial.
yuem.
emoran.
sanzire.
johneershaenus.
harla.
wymanusmarox.
demotha.
blabel.
parmelis.
chri.
ana.
azazaru.
mulwen.
jeita.
kella.
romer.
jalazari.
omyna.
ketri.
mamillariserab.
dorie.
islontarely.
reslayna.
rayah.


In [ ]:
for i in names:
    if i[:-1] in train:
        print(i)

In [ ]:
names[0][:-1]